In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
# # TODO: Fill in the `ROOT_PATH` with where you download the Assignment folder
ROOT_PATH = "/content/drive/MyDrive/DeepLearning_Project/"  # Replace with your directory to A3 folder
print(os.listdir(ROOT_PATH)) # Check the content of the path
os.chdir(ROOT_PATH) # cd into directory
print(os.listdir(".")) # Check the content of current folder

['yt_video_api.py', 'Wav2vec.md', 'models', 'predictions', 'tensorboard', 'data', 'Wav2vec-Copy1.ipynb', 'dataset_prep.py', 'base_model.md', 'logistic_regression.py', 'image.png', 'preprocessing.md', 'README.md', 'representation_learner.py', 'requirements.txt', 'GenerateDataset.ipynb', 'Wav2vec.ipynb', 'T5 Pretrained.ipynb', 'logistic_regression_weights.pkl', 'requirements.gdoc', '__pycache__']
['yt_video_api.py', 'Wav2vec.md', 'models', 'predictions', 'tensorboard', 'data', 'Wav2vec-Copy1.ipynb', 'dataset_prep.py', 'base_model.md', 'logistic_regression.py', 'image.png', 'preprocessing.md', 'README.md', 'representation_learner.py', 'requirements.txt', 'GenerateDataset.ipynb', 'Wav2vec.ipynb', 'T5 Pretrained.ipynb', 'logistic_regression_weights.pkl', 'requirements.gdoc', '__pycache__']


In [4]:
requirements = ROOT_PATH + "/requirements.txt"
%pip install -r {requirements}

  Using cached datasets-2.3.2-py3-none-any.whl (362 kB)
  Using cached evaluate-0.1.2-py3-none-any.whl (53 kB)
  Using cached huggingface_hub-0.8.1-py3-none-any.whl (101 kB)
  Using cached response-0.5.0-py3-none-any.whl (14 kB)
  Using cached transformers-4.12.5-py3-none-any.whl (3.1 MB)
  Using cached seqeval-1.2.2-py3-none-any.whl
  Using cached deepspeed-0.5.7-py3-none-any.whl
  Using cached tensorboard-2.7.0-py3-none-any.whl (5.8 MB)
  Using cached timm-0.4.12-py3-none-any.whl (376 kB)
  Using cached einops-0.8.0-py3-none-any.whl (43 kB)
  Using cached textdistance-4.6.2-py3-none-any.whl (31 kB)
  Using cached tokenizers-0.10.1.tar.gz (210 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached dill-0.3.5.1-py2.py3-none-any.whl (95 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
  Using cached sacremoses-0.1.1-py3-none-any.whl (897 kB)
  Using cached ninja-1.11.1.1-py2.py

In [5]:
!pip install evaluate
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 5.4 MB/s eta 0:00:00


In [4]:
import torch
from data.fetch_dataset_from_hf import fetch_dataset_from_huggingface
from datasets import load_dataset, Audio

# Load dataset

In [5]:
dataset = load_dataset("PolyAI/minds14", name="en-US", split="train")
print(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for PolyAI/minds14 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/PolyAI/minds14
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.w

Dataset({
    features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
    num_rows: 563
})


In [6]:
dataset = dataset.train_test_split(test_size=0.2) #split train set into train and test set

# Display data format

In [7]:
print(dataset["train"][0])
print(dataset["train"][2])
waveform_0 = dataset["train"][0]["audio"]
waveform_0

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/28aa727f91fee90575c34956bab09d1716cfaf460c6afcba86a10f04a7d58b83/en-US~BUSINESS_LOAN/602ba82b5f67b421554f643a.wav', 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/28aa727f91fee90575c34956bab09d1716cfaf460c6afcba86a10f04a7d58b83/en-US~BUSINESS_LOAN/602ba82b5f67b421554f643a.wav', 'array': array([ 0.00024414,  0.00024414,  0.        , ...,  0.        ,
        0.        , -0.00024414]), 'sampling_rate': 8000}, 'transcription': "I'd like some information about a business loan", 'english_transcription': "I'd like some information about a business loan", 'intent_class': 5, 'lang_id': 4}
{'path': '/root/.cache/huggingface/datasets/downloads/extracted/28aa727f91fee90575c34956bab09d1716cfaf460c6afcba86a10f04a7d58b83/en-US~DIRECT_DEBIT/602bae94bb1e6d0fbce92271.wav', 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/28aa727f91fee90575c34956bab09d1716cfaf460c6afcba86a10f04a7d58b83/en-US

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/28aa727f91fee90575c34956bab09d1716cfaf460c6afcba86a10f04a7d58b83/en-US~BUSINESS_LOAN/602ba82b5f67b421554f643a.wav',
 'array': array([ 0.00024414,  0.00024414,  0.        , ...,  0.        ,
         0.        , -0.00024414]),
 'sampling_rate': 8000}

In [8]:
dataset["train"].features

{'path': Value(dtype='string', id=None),
 'audio': Audio(sampling_rate=8000, mono=True, decode=True, id=None),
 'transcription': Value(dtype='string', id=None),
 'english_transcription': Value(dtype='string', id=None),
 'intent_class': ClassLabel(names=['abroad', 'address', 'app_error', 'atm_limit', 'balance', 'business_loan', 'card_issues', 'cash_deposit', 'direct_debit', 'freeze', 'high_value_payment', 'joint_account', 'latest_transactions', 'pay_bill'], id=None),
 'lang_id': ClassLabel(names=['cs-CZ', 'de-DE', 'en-AU', 'en-GB', 'en-US', 'es-ES', 'fr-FR', 'it-IT', 'ko-KR', 'nl-NL', 'pl-PL', 'pt-PT', 'ru-RU', 'zh-CN'], id=None)}

# Generate meaningful representations

In [11]:
dataset = dataset.remove_columns(["path", "transcription", "english_transcription", "lang_id"]) #removes unused columns

In [12]:
labels = dataset["train"].features["intent_class"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [13]:
id2label[str(1)]

'address'

In [15]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

def prep_function(dataset):
    audio_arrays = [x["array"] for x in dataset["audio"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000, truncation=True
    )
    return inputs

#encoded_dataset = preprocess_function(dataset, feature_extractor)
encoded_dataset = dataset.map(prep_function, remove_columns="audio", batched=True)
encoded_dataset = encoded_dataset.rename_column("intent_class", "label")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:363: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/113 [00:00<?, ? examples/s]

In [16]:
import numpy as np
import evaluate

def compute_metrics(eval_pred):
    accuracy = evaluate.load("accuracy")
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

# Finetune model

The Wav2Vec2ForSequenceClassification is essentially `context representation extractor + classifier` whereas the basic Wav2Vec2 does not have any classifier on top. So, the idea for now is to use the former model and finetune it on our hatespeech dataset. The finetuning uses the text too.

In [17]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base", num_labels=num_labels, label2id=label2id, id2label=id2label
)


Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
training_args = TrainingArguments(
    output_dir="my_awesome_mind_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

In [19]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
0,No log,2.635349,0.079646
1,No log,2.646133,0.053097


KeyboardInterrupt: 

# Predict new audios

In [ ]:

model = Wav2Vec2ForSequenceClassification.from_pretrained("superb/wav2vec2-base-superb-ks")
inputs = feature_extractor(dataset["train"][0]["audio_waveform"]["array"], sampling_rate=dataset["train"].features["audio_waveform"].sampling_rate, return_tensor="pt")

print(inputs)
inputs['input_values'] = torch.from_numpy(inputs['input_values'])
inputs['attention_mask'] = torch.from_numpy(inputs['attention_mask'][0]).unsqueeze(0)
with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_ids = torch.argmax(logits, dim=-1).item()
predicted_label = model.config.id2label[predicted_class_ids]
print(predicted_label)

target_label = model.config.id2label[0]
print(model.config.id2label)
inputs["labels"] = torch.tensor([model.config.label2id[target_label]])
loss = model(**inputs).loss
print(loss)

# Other shit

In [16]:
from huggingface_hub import login
login()

In [ ]:
from data.upload_dataset_to_hf import upload_dataset_to_huggingface

%cd data
upload_dataset_to_huggingface()

C:\Users\Damian Kopp\Documents\EPFL\MA2_DeepLearning\DeepLearning_Project\data
Largest audio ID: 1976


Casting to class labels:   0%|          | 0/1977 [00:00<?, ? examples/s]

Pushing split train to the Hub.


{'path': 'audio/sample_165.mp3', 'array': array([-3.05533376e-13,  1.42108547e-13, -4.54747351e-13, ...,
       -4.57438205e-07, -3.54410474e-07, -1.31835606e-07]), 'sampling_rate': 16000}
{'audio_waveform': {'path': 'audio/sample_165.mp3', 'array': array([-3.05533376e-13,  1.42108547e-13, -4.54747351e-13, ...,
       -4.57438205e-07, -3.54410474e-07, -1.31835606e-07]), 'sampling_rate': 16000}, 'hate_speech_score': 2.3, 'label': 0, 'text': 'fags are kiddie fuckers thats why, I used to support them, no more'}

{'path': 'audio/sample_1286.mp3', 'array': array([0., 0., 0., ..., 0., 0., 0.]), 'sampling_rate': 16000}
{'audio_waveform': {'path': 'audio/sample_1286.mp3', 'array': array([0., 0., 0., ..., 0., 0., 0.]), 'sampling_rate': 16000}, 'hate_speech_score': 1.74, 'label': 0, 'text': "Damn, that lefty city must STINK. Sweaty feminists wiping under their pendulous breasts with old Ariana Grande T-shirts. Obama's favorites drenched in every alley, offering sweet release to Democrat politici

Map:   0%|          | 0/1581 [00:00<?, ? examples/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
%cd ..

In [ ]:
spectrogram = librosa.feature.melspectrogram(y=audio_0, sr=sr)
print(spectrogram.shape)

# Convert to decibel scale (log scale)
spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)

# Plot the spectrogram
plt.figure(figsize=(10, 4))
librosa.display.specshow(spectrogram_db, sr=sr, x_axis='time', y_axis='mel')
plt.colorbar(format='%+2.0f dB')
plt.title('Mel Spectrogram')
plt.show()

In [ ]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import sounddevice as sd


def play_audio(_):
    sd.play(audio_0, sr)


# Create a play button
play_button = widgets.Button(description="Play")

# Attach the function to the button's click event
play_button.on_click(play_audio)

# Display the button below the cell
display(play_button)

# Model Instantiation
[https://huggingface.co/learn/audio-course/chapter3/classification](https://huggingface.co/learn/audio-course/chapter3/classification)

[Paper: AST: Audio Spectrogram Transformer](https://arxiv.org/pdf/2104.01778.pdf)